In [8]:
import os
import xarray as xr
import gcsfs
import numpy as np

# ─── CONFIG ────────────────────────────────────────────────────────────────────
GCS_URI_6H_13L = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"
OUT_6H_13L_ZARR = "ERA5_2020-2022_6h_5VAR_0.25.zarr"
OUT_6H_13L_NC = "ERA5_2020-2022_6h_5VAR_0.25.nc"
OUT_6H_13L_1deg_ZARR = "ERA5_2020-2022_6h_5VAR_1.0.zarr"
OUT_6H_13L_1deg_NC = "ERA5_2020-2022_6h_5VAR_1.0.nc"

# set start and end dates for data range. yyyy-mm-dd
start = "2020-01-01"
end = "2022-12-31"

VARS = [
    "2m_temperature",
    "mean_sea_level_pressure",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "total_precipitation_6hr"
]

def open_gcs_zarr(uri: str, project: str = None):
    fs = gcsfs.GCSFileSystem(project=project)
    return xr.open_zarr(fs.get_mapper(uri), consolidated=True)




# 1) load & time‐subset 6h/13l for 2020-2022
ds = open_gcs_zarr(GCS_URI_6H_13L)
ds2020_22 = ds.sel(time=slice(start, end))

# 1b) keep only the five surface variables
ds2020_22 = ds2020_22[VARS]

times = ds2020_22.time.values
start = np.min(times)
end   = np.max(times)
print(f"Dataset contains time from {start} to {end}")

# use the actual coord names in your ds:
lon_coord = "longitude"    if "longitude"    in ds2020_22.coords else "lon"
lat_coord = "latitude"     if "latitude"     in ds2020_22.coords else "lat"

# only save US subset
us_lon_min, us_lon_max = 235.0, 294.0   # [0–360] U.S. longitudes
us_lat_min, us_lat_max =  24.0,  50.0   # U.S. latitudes
ds2020_22 = ( ds2020_22.sortby(lon_coord)
    .sortby(lat_coord)
    .sel({
        lon_coord: slice(us_lon_min, us_lon_max),
        lat_coord: slice(us_lat_min, us_lat_max)
    })
)


# 2) write out the U.S. subsets

ds2020_22.to_zarr(OUT_6H_13L_ZARR, mode="w")
print("US 2020-2022 0.25 degree ZArr →", OUT_6H_13L_ZARR)
ds2020_22.to_netcdf(OUT_6H_13L_NC)
print("US 2020-2022 0.25 degree NetCDF4 →", OUT_6H_13L_NC)

ds2020_22_1deg = ds2020_22.coarsen(
    {lat_coord: 4, lon_coord: 4},
    boundary="trim").mean()


ds2020_22_1deg.to_zarr(OUT_6H_13L_1deg_ZARR, mode="w")
print("US 2020-2022 1.0 degree ZArr →", OUT_6H_13L_1deg_ZARR)
ds2020_22_1deg.to_netcdf(OUT_6H_13L_1deg_NC)
print("US 2020-2022 1.0 degree NetCDF4 →", OUT_6H_13L_1deg_NC)


Dataset contains time from 2020-01-01T00:00:00.000000000 to 2022-12-31T18:00:00.000000000
US 2020-2022 0.25 degree ZArr → ERA5_2020-2022_6h_5VAR_0.25.zarr
US 2020-2022 0.25 degree NetCDF4 → ERA5_2020-2022_6h_5VAR_0.25.nc
US 2020-2022 1.0 degree ZArr → ERA5_2020-2022_6h_5VAR_1.0.zarr
US 2020-2022 1.0 degree NetCDF4 → ERA5_2020-2022_6h_5VAR_1.0.nc


Then gather arrival data

In [9]:


# ─── 1) Inspect your ERA5 dataset ──────────────────────────────────────────────
ERA5_STORE = "ERA5_2020-2022_6h_5VAR_1.0.zarr"
ds = xr.open_zarr(ERA5_STORE)

print("=== ERA5 DATASET SUMMARY ===")
print(ds)                        # full summary: dims, coords, data_vars
print("\nVariables in ds.data_vars:")
for name, var in ds.data_vars.items():
    print(f" • {name:30s} dims={var.dims}   shape={tuple(var.shape)}")

print("\nCoordinates in ds.coords:")
for coord in ds.coords:
    print(" •", coord, "=", ds.coords[coord].values[:5], "…")



=== ERA5 DATASET SUMMARY ===
<xarray.Dataset> Size: 135MB
Dimensions:                  (time: 4384, latitude: 26, longitude: 59)
Coordinates:
  * latitude                 (latitude) float32 104B 24.38 25.38 ... 48.38 49.38
  * longitude                (longitude) float32 236B 235.4 236.4 ... 293.4
  * time                     (time) datetime64[ns] 35kB 2020-01-01 ... 2022-1...
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 27MB dask.array<chunksize=(1, 26, 59), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 27MB dask.array<chunksize=(1, 26, 59), meta=np.ndarray>
    2m_temperature           (time, latitude, longitude) float32 27MB dask.array<chunksize=(1, 26, 59), meta=np.ndarray>
    mean_sea_level_pressure  (time, latitude, longitude) float32 27MB dask.array<chunksize=(1, 26, 59), meta=np.ndarray>
    total_precipitation_6hr  (time, latitude, longitude) float32 27MB dask.array<chunksize=(1, 26, 59), meta=np.ndarray>

augment arrival data with weather data

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
#!pip install pyproj
from pyproj import geod

# ─── CONFIG ───────────────────────────────────────────────────────────────────
ERA5_STORE   = "ERA5_2020-2022_6h_5VAR_0.25.zarr"
MASTER_COORD = "Datasets/T_MASTER_CORD.csv"
ARRIVAL_DIR  = "Datasets/Arrival_Statistics/2020-2022"
OUTPUT_DIR   = "Datasets/Arrival_With_Weather/2020-2022"

VARS = [
    "2m_temperature",
    "mean_sea_level_pressure",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "total_precipitation_6hr"
]



os.makedirs(OUTPUT_DIR, exist_ok=True)

# ─── 1) load ERA5 & rename dims ───────────────────────────────────────────────
ds = xr.open_zarr(ERA5_STORE, consolidated=True)

if "latitude" in ds.dims and "longitude" in ds.dims:
    ds = ds.rename({"latitude": "lat", "longitude": "lon"})

print("dims before subsetting:", ds.dims)

# ─── 1b) wrap & sort lon/lat, then slice just over the U.S. ────────────────
# Only do this slice if flight data contains only Domestic U.S. Flights and weather data contains the world.
#us_lon_min, us_lon_max = 235.0, 294.0   # [0–360] U.S. longitudes
#us_lat_min, us_lat_max =  24.0,  50.0   # U.S. latitudes

#ds = (ds.assign_coords(lon=((ds.lon + 360) % 360)).sortby("lon").sortby("lat").sel(lon=slice(us_lon_min, us_lon_max),lat=slice(us_lat_min, us_lat_max)))
#print("dims after subsetting:", ds.dims)

# ─── 2) build your fast lookup arrays ────────────────────────────────────────
time_index = ds["time"].to_index()
lat_vals    = ds["lat"].values
lon_vals    = ds["lon"].values
var_arrays = {var: ds[var].data.compute()  for var in VARS}


# ─── 3) load master coords ───────────────────────────────────────────────────
mc = (
    pd.read_csv(MASTER_COORD, dtype=str)
      .set_index("AIRPORT_SEQ_ID")[["LATITUDE","LONGITUDE"]]
      .astype(float)
)

# ─── 4) helpers ─────────────────────────────────────────────
def parse_hhmm(x):
    if pd.isna(x) or not str(x).strip():
        return pd.NaT
    s = str(int(float(x))).zfill(4)
    return pd.Timedelta(hours=int(s[:2]), minutes=int(s[2:]))



# ─── then your make_weather_lookup becomes ─────────────────────────────────
def make_weather_lookup(keys: pd.DataFrame) -> pd.DataFrame:
    sub = keys.copy()
    sub["lat"] = sub["AirportSeqID"].map(mc["LATITUDE"])
    sub["lon"] = sub["AirportSeqID"].map(mc["LONGITUDE"]) % 360.0
    
    
    #sub = sub.dropna(subset=["Datetime","lat","lon"])
    if sub.empty:
        return pd.DataFrame(columns=VARS,
                            index=pd.MultiIndex.from_arrays([[],[]],
                                                           names=["AirportSeqID","Datetime"]))

    # 1) nearest‐time
    t_idx = time_index.get_indexer(sub["Datetime"], method="nearest")

    # 2) nearest‐lat/lon
    sub_lat = sub["lat"].to_numpy()
    sub_lon = sub["lon"].to_numpy()
    l_idx = np.abs(lat_vals[None,:] - sub_lat[:,None]).argmin(axis=1)
    o_idx = np.abs(lon_vals[None,:] - sub_lon[:,None]).argmin(axis=1)

    # 3) pull out each VAR from our preloaded var_arrays
    out = {}
    for var in VARS:
        arr = var_arrays[var]   # pure numpy now
        out[var] = arr[t_idx, l_idx, o_idx]

    # 4) assemble a DataFrame and re‑index
    df_lkp = pd.DataFrame(out, index=sub.index)
    df_lkp.index = pd.MultiIndex.from_frame(
        sub[["AirportSeqID","Datetime"]],
        names=["AirportSeqID","Datetime"]
    )
    return df_lkp


# unused code to extract points along expected route for further prediction
def make_route_lookup(df: pd.DataFrame, spacing_miles=100) -> pd.DataFrame:
    """
    For each flight (row of df), sample points every ~spacing_miles along the
    great‐circle from origin→dest at the departure time. Then compute
    mean/min/max for each VAR along that route.
    """
    records = []
    for idx, row in df.iterrows():
        orig = row["OriginAirportSeqID"]
        dest = row["DestAirportSeqID"]
        t0   = row["DepDatetime"]

        lat0, lon0 = mc.loc[orig]
        lat1, lon1 = mc.loc[dest]
        # total distance in meters:
        _,_,dist_m = geod.inv(lon0, lat0, lon1, lat1)
        # number of segments
        nseg = max(1, int(dist_m / (spacing_miles*1609.34)))
        # intermediate points (excluding endpoints)
        pts = geod.npts(lon0, lat0, lon1, lat1, nseg-1)
        # build full list including endpoints
        all_pts = [(lat0,lon0)] + [(lat,lon) for lon,lat in pts] + [(lat1,lon1)]

        t_idx = time_index.get_indexer([t0], method="nearest")[0]
        vals = {var: [] for var in VARS}

        for lat, lon in all_pts:
            # nearest grid
            i_lat = np.abs(lat_vals - lat).argmin()
            i_lon = np.abs(lon_vals - (lon%360)).argmin()
            for var in VARS:
                vals[var].append(var_arrays[var][t_idx, i_lat, i_lon])

        # compute summaries
        rec = {"AirportSeqID": orig, "Datetime": t0}
        for var, arr in vals.items():
            rec[f"Route_{var}_mean"] = np.nanmean(arr)
            rec[f"Route_{var}_min"]  = np.nanmin(arr)
            rec[f"Route_{var}_max"]  = np.nanmax(arr)
        records.append(rec)

    if not records:
        return pd.DataFrame([], columns=[f"Route_{v}_{s}" for v in VARS for s in ("mean","min","max")],
                            index=pd.MultiIndex(levels=[[],[]], codes=[[],[]],
                                                names=["AirportSeqID","Datetime"]))

    df_route = pd.DataFrame(records)
    idx = pd.MultiIndex.from_frame(df_route[["AirportSeqID","Datetime"]],
                                   names=["AirportSeqID","Datetime"])
    df_route.index = idx
    return df_route.drop(columns=["AirportSeqID","Datetime"])






dims before subsetting: FrozenMappingWarningOnValuesAccess({'time': 4384, 'lat': 105, 'lon': 237})


In [11]:
for fn in sorted(os.listdir(ARRIVAL_DIR)):
    if not fn.lower().endswith(".csv"):
        continue
    print("→ augmenting", fn)
    df = pd.read_csv(os.path.join(ARRIVAL_DIR, fn), dtype=str)

    # build timestamps
    df["FlightDate"]   = pd.to_datetime(df["FlightDate"], format="%Y-%m-%d", errors="coerce")
    df["DepDelta"]     = df["DepTime"].apply(parse_hhmm)
    df["ArrDelta"]     = df["ArrTime"].apply(parse_hhmm)
    df["DepDatetime"]  = df["FlightDate"] + df["DepDelta"]
    df["ArrDatetime"]  = df["FlightDate"] + df["ArrDelta"]

    # origin & dest lookups
    orig = ( df[["OriginAirportSeqID","DepDatetime"]]
             .dropna().drop_duplicates()
             .rename(columns={"OriginAirportSeqID":"AirportSeqID","DepDatetime":"Datetime"}) )
    dest = ( df[["DestAirportSeqID","ArrDatetime"]]
             .dropna().drop_duplicates()
             .rename(columns={"DestAirportSeqID":"AirportSeqID","ArrDatetime":"Datetime"}) )

    orig_lkp  = make_weather_lookup(orig).rename(columns=lambda c: f"Origin_{c}")
    dest_lkp  = make_weather_lookup(dest).rename(columns=lambda c: f"Dest_{c}")

    # route lookups
    #route_df  = df[["OriginAirportSeqID","DepDatetime"]].dropna().drop_duplicates()
    #route_df  = route_df.rename(columns={"OriginAirportSeqID":"AirportSeqID","DepDatetime":"Datetime"})
    #route_lkp = make_route_lookup(route_df, spacing_miles=100)

    # merge everything back
    df = (df
          .merge(orig_lkp,  left_on=["OriginAirportSeqID","DepDatetime"], right_index=True, how="left")
          .merge(dest_lkp,  left_on=["DestAirportSeqID","ArrDatetime"],   right_index=True, how="left")
          #.merge(route_lkp, left_on=["OriginAirportSeqID","DepDatetime"], right_index=True, how="left")
        )

    # cleanup & save
    df.drop(columns=["DepDelta","ArrDelta","DepDatetime","ArrDatetime"], errors="ignore", inplace=True)
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    df.dropna(axis=1, how="all", inplace=True)

    out = os.path.join(OUTPUT_DIR, fn)
    df.to_csv(out, index=False)
    print("   saved →", out)

→ augmenting On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_1.csv


C:\Users\ebror\AppData\Local\Temp\ipykernel_31592\3010907270.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


   saved → Datasets/Arrival_With_Weather/2020-2022\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_1.csv
→ augmenting On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_10.csv


C:\Users\ebror\AppData\Local\Temp\ipykernel_31592\3010907270.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


   saved → Datasets/Arrival_With_Weather/2020-2022\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_10.csv
→ augmenting On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_11.csv


...

   saved → Datasets/Arrival_With_Weather/2020-2022\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_7.csv
→ augmenting On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_8.csv


C:\Users\ebror\AppData\Local\Temp\ipykernel_31592\3010907270.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


   saved → Datasets/Arrival_With_Weather/2020-2022\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_8.csv
→ augmenting On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_9.csv


C:\Users\ebror\AppData\Local\Temp\ipykernel_31592\3010907270.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


   saved → Datasets/Arrival_With_Weather/2020-2022\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_9.csv


In [12]:
# ─── 2) Inspect one of your cleaned arrivals CSVs ──────────────────────────────
ARRIVAL_DIR = "Datasets/Arrival_With_Weather/2020-2022"
# pick the first CSV in the folder
fn = sorted([f for f in os.listdir(ARRIVAL_DIR) if f.lower().endswith(".csv")])[0]
df = pd.read_csv(os.path.join(ARRIVAL_DIR, fn), parse_dates=["FlightDate"])

print("\n=== SAMPLE ARRIVAL CSV:", fn, "===\n")
print("Columns:", list(df.columns))
print("\nFirst 5 rows:")
print(df.head())

C:\Users\ebror\AppData\Local\Temp\ipykernel_31592\3948918952.py:5: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(ARRIVAL_DIR, fn), parse_dates=["FlightDate"])



=== SAMPLE ARRIVAL CSV: On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_1.csv ===

Columns: ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'C

Now run preprocessing to merge and format the arrival data for training